In [12]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFDirectoryLoader
from llama_index.core.node_parser import SentenceSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, PodSpec
import os
import logging
import sys
from llama_index.core import (
    VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext
)
from llama_index.vector_stores.pinecone import PineconeVectorStore
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
import torch
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core import ServiceContext
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
documents = SimpleDirectoryReader("./Data").load_data()

Multiple definitions in dictionary at byte 0x570ea for key /Type
Multiple definitions in dictionary at byte 0x570f9 for key /Subtype
Multiple definitions in dictionary at byte 0x5710d for key /BBox
Multiple definitions in dictionary at byte 0x57159 for key /Resources
Multiple definitions in dictionary at byte 0x59cd7 for key /Type
Multiple definitions in dictionary at byte 0x59ce6 for key /Subtype
Multiple definitions in dictionary at byte 0x59d09 for key /BBox
Multiple definitions in dictionary at byte 0x59d55 for key /Resources


In [55]:
llm.complete("hi")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   19857.91 ms
llama_print_timings:      sample time =       0.97 ms /    10 runs   (    0.10 ms per token, 10256.41 tokens per second)
llama_print_timings: prompt eval time =    3790.32 ms /     5 tokens (  758.06 ms per token,     1.32 tokens per second)
llama_print_timings:        eval time =    1297.76 ms /     9 runs   (  144.20 ms per token,     6.94 tokens per second)
llama_print_timings:       total time =    5105.77 ms /    14 tokens


CompletionResponse(text=' Hello! How can I assist you today?', additional_kwargs={}, raw={'id': 'cmpl-8814763e-9271-4cc5-b579-900b31d78102', 'object': 'text_completion', 'created': 1712263372, 'model': '/Users/adityatodi/Library/Caches/llama_index/models/mistral-7b-instruct-v0.1.Q8_0.gguf', 'choices': [{'text': ' Hello! How can I assist you today?', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 64, 'completion_tokens': 9, 'total_tokens': 73}}, logprobs=None, delta=None)

In [8]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q8_0.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/adityatodi/Library/Caches/llama_index/models/mistral-7b-instruct-v0.1.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_

In [9]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
Use pytorch device_name: mps
Use pytorch device_name: mps


In [15]:
load_dotenv()

# initialize connection to pinecone
api_key = os.environ["PINECONE_API_KEY"]
index_name = "os-helper"
pc = Pinecone(api_key=api_key)

# connect to the index
pinecone_index = pc.Index(index_name)

In [47]:
pinecone_index.as_chat_engine()

AttributeError: 'Index' object has no attribute 'as_chat_engine'

In [37]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

In [45]:
import langchain.vectorstores as vs

In [46]:
pc_interface = vs.Pinecone.from_existing_index(
    index_name, 
    embedding=embed_model
)

In [39]:
vector_store.as_retriever(search_kwargs={"k": 4})

AttributeError: 'PineconeVectorStore' object has no attribute 'as_retriever'

In [50]:
vector_store = PineconeVectorStore(pinecone_index)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, embed_model=embed_model)

In [16]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
# setup our storage (vector db)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

# setup our service context
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    chunk_overlap=64,
    llm=llm,
    embed_model=embed_model
)

/var/folders/41/w6fqcv917fb6nskgcyj7x_800000gn/T/ipykernel_13995/2085093085.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [17]:
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, service_context=service_context)

Upserted vectors:   0%|          | 0/952 [00:00<?, ?it/s]

In [27]:
type(index)

llama_index.core.indices.vector_store.base.VectorStoreIndex

In [22]:
query_engine = index.as_query_engine()
response = query_engine.query("How is page fault handled in exokernel?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   19857.91 ms
llama_print_timings:      sample time =       8.64 ms /    95 runs   (    0.09 ms per token, 10996.64 tokens per second)
llama_print_timings: prompt eval time =    4324.92 ms /   348 tokens (   12.43 ms per token,    80.46 tokens per second)
llama_print_timings:        eval time =    8084.99 ms /    94 runs   (   86.01 ms per token,    11.63 tokens per second)
llama_print_timings:       total time =   12548.01 ms /   442 tokens


In [23]:
print(response)

 In an Exokernel, page faults are handled by the kernel asking an application to give up a page. The application decides which page to give up and communicates this decision to the kernel through an upcall (kernel → app). The kernel then releases the page and updates the TLB entries. If the kernel needs to force a page fault, it can do so by sending a signal to the application or by modifying the memory management policies of the application.


In [24]:
pinecone_index1 = pc.Index(index_name)

In [30]:
vector_store1 = PineconeVectorStore(pinecone_index=pinecone_index1)

# Create a StorageContext
storage_context1 = StorageContext.from_defaults(vector_store=vector_store)

# Create a VectorStoreIndex
index1 = VectorStoreIndex([], storage_context=storage_context1, service_context=service_context)
query_engine = index1.as_query_engine()

In [33]:
print(query_engine.query("In FlexSC paper, how did the authors calculate the direct & indirect cost?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   19857.91 ms
llama_print_timings:      sample time =      20.63 ms /   225 runs   (    0.09 ms per token, 10907.50 tokens per second)
llama_print_timings: prompt eval time =    3748.66 ms /   291 tokens (   12.88 ms per token,    77.63 tokens per second)
llama_print_timings:        eval time =   19183.35 ms /   224 runs   (   85.64 ms per token,    11.68 tokens per second)
llama_print_timings:       total time =   23275.76 ms /   515 tokens


 The authors of the FlexSC paper calculated the direct and indirect costs in the following way:

1. Direct cost: The direct cost refers to the mode-switching overhead that occurs when a system call is made from one thread to another. To calculate this, the authors measured the time it took for a thread to switch modes (i.e., from user space to kernel space and vice versa) using a hardware timer. They then used this measurement to estimate the cost of making a system call from one thread to another.
2. Indirect cost: The indirect cost refers to the overhead associated with the processor structure pollution that occurs when a system call is made from one thread to another. To calculate this, the authors measured the time it took for the processor to switch from one thread to another using a hardware timer. They then used this measurement to estimate the cost of making a system call from one thread to another.

The authors also mentioned that they used a combination of these two costs to 

In [62]:
response.source_nodes[0].node.metadata['file_name']

<class 'llama_index.core.schema.NodeWithScore'>
<class 'llama_index.core.schema.NodeWithScore'>
